In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import boto3
import pandas as pd
import urllib

#setup
CREDENTIALS_FILE = './credentials.csv'
credentials = pd.read_csv(CREDENTIALS_FILE).to_dict('records')[0]
aws_access_key_id = credentials['Access key ID']
aws_secret_access_key = credentials['Secret access key']

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    endpoint_url = endpoint_url,
    region_name = region_name
)


In [ ]:
#create qualification requirement

question = '''<QuestionForm xmlns='http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionForm.xsd'>
<Question>
      <QuestionIdentifier>password_test</QuestionIdentifier>
      <DisplayName>Question</DisplayName>
      <IsRequired>true</IsRequired>
      <QuestionContent>
        <Text> Please enter password </Text>
      </QuestionContent>
      <FreeTextAnswer>
        <Constraints>
          <Length minLength="2" />
          <AnswerFormatRegex regex="\S" errorText="The content cannot be blank."/>
        </Constraints>
      </FreeTextAnswer>
</Question> 
</QuestionForm>
'''

answer = '''<AnswerKey xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/AnswerKey.xsd">

</AnswerKey>
'''

qual_response = mturk.create_qualification_type(
                        Name='Enter password',
                        Keywords='test, qualification, password',
                        Description='Enter the correct password to access the questions',
                        QualificationTypeStatus='Active',
                        Test=question,
                        AnswerKey=answer,
                        TestDurationInSeconds=300)

print(qual_response['QualificationType']['QualificationTypeId'])

In [ ]:
#create hit type
one_minute = 60 # seconds
one_hour = 60 * one_minute
one_day = 24 * one_hour

hit_type_response = client.create_hit_type(
    AutoApprovalDelayInSeconds = 10 * one_minute,
    AssignmentDurationInSeconds = 5 * one_minute,
    Reward = '0.20',
    Title = 'Emotion in GIF',
    Keywords = 'emotion, GIF, detection, VAD',
    Description = 'Choose a GIF that best describes the image shown above',
    QualificationRequirements = [
      {
          'QualificationTypeId': '00000000000000000071',
          'Comparator': 'In',
          'LocaleValues': [
            {
                'Country': 'TW'
            },
            {
                'Country': 'US'
            },
          ],
          'RequiredToPreview': True,
          'ActionsGuarded': 'PreviewAndAccept'
      },
    ]
)

In [ ]:
#hit for vad
hit_type_id = hit_type_response['HITTypeId']
#for...
params = 'gifA=' + urllib.parse.quote('0ffa3c2efd2b21b19b846888670a8e36') + '&amp;' 'gifB=' + urllib.parse.quote('3a392f38bf5acc872a99a26bcc3c7ecc') + '&amp;''gifC=' + urllib.parse.quote('3b9eb91e33f9860c2923cfab6aa1d958') + '&amp;''gifD=' + urllib.parse.quote('4a130081ea119837ae185fe007f746cf') 
#D
question=f'''<?xml version="1.0" encoding="UTF-8"?>
    <ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
    <ExternalURL>https://doraemon.iis.sinica.edu.tw/gif-vad/test/templates/ques-D.html?{params}</ExternalURL>
    <FrameHeight>800</FrameHeight>
    </ExternalQuestion>'''

response = client.create_hit_with_hit_type(
      HITTypeId = hit_type_id,
      MaxAssignments = 3,
      LifetimeInSeconds = 10 * one_minute,
      Question = question,
      RequesterAnnotation = "D"
  )
print("done")

In [ ]:
#fetch results

#list_hits_response = client.list_hits()
#hit_id = list_hits_response['HITs'][0]
#response = client.list_assignments_for_hit(HITId=hit_id)
hits_paginator = client.get_paginator('list_hits')
assignments_paginator = client.get_paginator('list_assignments_for_hit')
ans = ""
for hits in hits_paginator.paginate():
  for hit in hits['HITs']:
    for assignments in assignments_paginator.paginate(HITId=hit['HITId']):
      for assignment in assignments['Assignments']:
        print(assignment['Answer'])